In [5]:
import numpy as np
L = list(np.loadtxt('syj_allnode_paper.txt'))    
nodelist = list(np.loadtxt('syj_nodelist_paper.txt'))
input_matlabinfo = 'syj_intervals_paper.txt'
distmat = np.loadtxt('syj_distmat_paper.txt')

###############################################################################        
def d1_info(filename):
    import json  
    f = open(filename,'r'); l = len(f.readlines())
    f = open(filename,'r')
    
    s = []
    for i in range(l):
        s.append(f.readline())
        
    # dim1 info
    d1s = []    
    for i in range(l):
        if s[i] == 'Dimension: 1\n':
            start_inx = i+1
    for j in range(start_inx,l):        
        if s[j] == 'Dimension: 2\n' or s[j] == '\n':
            end_inx = j
            break
    for k in range(start_inx,end_inx):        
            d1s.append(s[k])
            
    # get cycle info      
    c = []        
    for i in range(len(d1s)):
        ci_int,ci = d1s[i].split(': ')  
        ci = ci.split('\n')[0]
        ci_edge = ci.split(' + ')
        for j in range(len(ci_edge)):
            if ci_edge[j][0] == '-':
                ci_edge[j] = json.loads(ci_edge[j][1:])
                ci_edge[j] = ci_edge[j][::-1]
            else:
                ci_edge[j] = json.loads(ci_edge[j])
        ci_node = []
        for k in range(len(ci_edge)):
            for node in ci_edge[k]:
                if not(node in ci_node):
                    ci_node.append(node)
        ci_node.sort()
        c.append([ci_node,ci_int,ci_edge])
    return d1s,c    
###############################################################################        
def pit_to_note(pit):
    if pit == 349:
        note = 'F4 Tae'
    elif pit == 262:
        note = 'C4 Nam'
    elif pit == 233:
        note = 'A#3 Im'
    elif pit == 311:
        note = 'D#4 Hwang'
    elif pit == 208:
        note = 'G#3 Jung'
    elif pit == 466:
        note = 'A#4 Im'
    elif pit == 415:
        note = 'G#4 Jung'
    elif pit == 622:
        note = 'D#5 Hwang'
    elif pit == 523:
        note = 'C5 Nam'
    elif pit == 699:
        note = 'F5 Tae'
    elif pit == 278:
        note = 'C#4 Nam'
    return note
###############################################################################
def nodeID_to_music(l): 
    node_in_music = []
    for i in range(len(l)):
        pit = int( l[i] )         
        note = pit_to_note(pit)
        dur = round( ( l[i]-int(l[i]) ) *100 )
        node_in_music.append((note,dur))
    return node_in_music
###############################################################################
def find_cycles(s,c,e):
    #############################################
    L_pit = [int(L[i]) for i in range(len(L))]
    
    # build graph
    import networkx as nx
    G = nx.Graph()
    G.add_nodes_from(L)
    for i in range(len(L)-1):
        if L[i] != L[i+1]:
            e0 = L[i]
            e1 = L[i+1]
            if G.has_edge(e0,e1):
                G[e0][e1]['weight'] += 1
            else:
                G.add_edge(e0,e1,weight=1)            
    ##################################################
    for k in range(len(c)): # check for each cycle
        cycle_found = 0
                
        # find nodes in each cycle
        node_in_cycle = []
        for node in c[k]:
            node_in_cycle.append(nodelist[node])
        print('\n********************\n','barcode: ',s[k])
        print(' matlab_cycle',k+1,': ',node_in_cycle,'\n','node id:',c[k])
        # convert node_in_cycle to music_node
        music_node = nodeID_to_music(node_in_cycle)
        print(' music node:', music_node)
           
        
        # find edge weights
        weight = 0
        for edge in e[k]:
            e0 = nodelist[edge[0]]
            e1 = nodelist[edge[1]]
            edge_dist = distmat[edge[0]][edge[1]]
            if G.has_edge(e0,e1):
                print('weight', edge,':', G[e0][e1]['weight'],'--------- edge_dist = ',edge_dist)
                weight += G[e0][e1]['weight']
            else:
                print('there is no edge',edge,'--------- edge_dist = ',edge_dist)
        avgw = weight/len(e[k])
        print('sum weight = ',weight )
        print('avg weight = ',avgw )
        
          
        
        # compare (consecutive) nodes in L with nodes in each cycle
        
        for i in range(len(L)):
            if set(L[i:i+len(node_in_cycle)]) == set(node_in_cycle):
                cycle_found = 1
                print('   ------------\n   music data:',L[i:i+len(node_in_cycle)],'\n','  node id:',[nodelist.index(L[v]) for v in range(i,i+len(node_in_cycle))])
                node_in_music = L[i:i+len(node_in_cycle)]
                # convert to music: ( note , duration )    
                for l in range(len(node_in_music)):
                    pit = int( node_in_music[l] )         
                    note = pit_to_note(pit)
                    # duration of the cycle
                    dur = round( ( node_in_music[l]-int(node_in_music[l]) ) *100 )
                    print('   (',note,',',dur,')')

        if cycle_found == 0:
            print('   NOT found in music data')
            
###############################################################################
        
s,c_info = d1_info(input_matlabinfo)
c = []
e = []
for i in range(len(c_info)):
    c.append(c_info[i][0])
    e.append(c_info[i][2])
find_cycles(s,c,e)







********************
 barcode:  [0.14100000000000001, 0.165): [0,20] + [0,6] + [6,19] + [19,20]

 matlab_cycle 1 :  [349.06, 311.06, 349.02, 415.06] 
 node id: [0, 6, 19, 20]
 music node: [('F4 Tae', 6), ('D#4 Hwang', 6), ('F4 Tae', 2), ('G#4 Jung', 6)]
weight [0, 20] : 13 --------- edge_dist =  0.076923076923077
weight [0, 6] : 24 --------- edge_dist =  0.041666666666667
weight [6, 19] : 10 --------- edge_dist =  0.1
weight [19, 20] : 7 --------- edge_dist =  0.142857142857143
sum weight =  54
avg weight =  13.5
   ------------
   music data: [349.02, 415.06, 311.06, 349.06] 
   node id: [19, 20, 6, 0]
   ( F4 Tae , 2 )
   ( G#4 Jung , 6 )
   ( D#4 Hwang , 6 )
   ( F4 Tae , 6 )
   ------------
   music data: [349.02, 415.06, 311.06, 349.06] 
   node id: [19, 20, 6, 0]
   ( F4 Tae , 2 )
   ( G#4 Jung , 6 )
   ( D#4 Hwang , 6 )
   ( F4 Tae , 6 )

********************
 barcode:  [0.165, 0.17400000000000002): [14,15] + [5,6] + -[5,15] + [6,14]

 matlab_cycle 2 :  [233.06, 311.06, 233.02,